In [96]:
import pandas

pandas.set_option("display.max_rows", 100)
df = pandas.read_csv("./1.4.csv")  # Tuesday's changes included
df

,name,years,genres,rating,description,stars,votes,run_time,directors,dupe_counts
0,Blood Red Sky,(2021),"['Action', 'Horror', 'Thriller']",6.1,A woman with a mysterious illness is forced in...,"['Peri Baumeister', 'Carl Anton Koch', 'Alexan...","21,062",121.0,['Peter Thorwarth'],1
1,Masters of the Universe: Revelation,(2021– ),"['Animation', 'Action', 'Adventure']",5.0,The war for Eternia begins again in what may b...,"['Chris Wood', 'Sarah Michelle Gellar', 'Lena ...","17,870",25.0,[],1
2,The Walking Dead,(2010–2022),"['Drama', 'Horror', 'Thriller']",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,"['Andrew Lincoln', 'Norman Reedus', 'Melissa M...","885,805",44.0,[],1
3,Rick and Morty,(2013– ),"['Animation', 'Adventure', 'Comedy']",9.2,An animated series that follows the exploits o...,"['Justin Roiland', 'Chris Parnell', 'Spencer G...","414,849",23.0,[],1
4,Army of Thieves,(2021),"['Action', 'Crime', 'Horror']",NaN,"A prequel, set before the events of Army of th...","['Matthias Schweighöfer', 'Nathalie Emmanuel',...",0,NaN,['Matthias Schweighöfer'],1
...,...,...,...,...,...,...,...,...,...,...
9311,The Last Kingdom,(2015– ),"['Action', 'Drama', 'History']",NaN,NaN,[],0,NaN,['Anthony Philipson'],16
9312,Totenfrau,(2022– ),"['Drama', 'Thriller']",NaN,NaN,"['Felix Klare', 'Romina Küper', 'Anna Maria Mü...",0,NaN,['Nicolai Rohde'],1
9313,Heart of Invictus,(2022– ),"['Documentary', 'Sport']",NaN,NaN,['Prince Harry'],0,NaN,['Orlando von Einsiedel'],1
9314,The Imperfects,(2021– ),"['Adventure', 'Drama', 'Fantasy']",NaN,NaN,"['Morgan Taylor Campbell', 'Iñaki Godoy', 'Rhi...",0,NaN,['Jovanka Vuckovic'],3


Parse `years` with RegEx 
- split into a start and end field
- leave end as NaN if still running
- leave both as NaN if source is NaN
- correctly handle shows that only lasted one year

In [97]:
years = df["years"].str.extract(r"\((?P<start>\d{4})(?: \w|(?:–(?P<end> |\d{4}))?\)$)")
years["source"] = df["years"]
years

,start,end,source
0,2021,NaN,(2021)
1,2021,,(2021– )
2,2010,2022,(2010–2022)
3,2013,,(2013– )
4,2021,NaN,(2021)
...,...,...,...
9311,2015,,(2015– )
9312,2022,,(2022– )
9313,2022,,(2022– )
9314,2021,,(2021– )


In [98]:
# Any remaining unparsable sources?
years[~years["start"].astype(str).str.isdecimal()]["source"].value_counts()
# After my first attempt, I hadn't accounted for values that includes roman numerals or other note AS WELL as a year
# Now, the only unparasble values don't have years in them
# Note that .value_counts() doesn't count `NaN` values in the `source` column

(II)            20
(I)             17
(III)           15
(TV Movie)       9
(VII)            7
(TV Special)     5
(IV)             4
(V)              3
(XI)             2
(Video)          2
(XXIII)          1
(XLI)            1
(VI)             1
(XIII)           1
(VIII)           1
(IX)             1
(XII)            1
Name: source, dtype: int64

In [99]:
# Replace values in the `end` column
# If it's `NaN`, it only lasted a year. Set to `start`
# If it's `" "`, it's still running. Set to `None`
from numpy import NaN

years["end"].fillna(years["start"], inplace=True)
years["end"].replace(" ", NaN, inplace=True)
years

,start,end,source
0,2021,2021,(2021)
1,2021,NaN,(2021– )
2,2010,2022,(2010–2022)
3,2013,NaN,(2013– )
4,2021,2021,(2021)
...,...,...,...
9311,2015,NaN,(2015– )
9312,2022,NaN,(2022– )
9313,2022,NaN,(2022– )
9314,2021,NaN,(2021– )


In [100]:
df["start_year"] = years["start"]
df["end_year"] = years["end"]
df.drop(columns=["years", "dupe_counts"], inplace=True)

Will finish tomorrow

In [101]:
df.to_csv("./1.5.csv", index=False)